<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-20 12:37:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-20 12:37:19 (34.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-20 12:37:23--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-09-20 12:37:23 (106 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 Ignition Knock (Detonation) Sensor-Senso fits 01-06 BMW 325Ci 2.5L-L6 http://t.co/gBVDNczjoU http://t.co/c211HISe0R 

---

target:1 (real disaster)
Text
 Around 10 injured in explosion in chemical park in western Germany... http://t.co/XBznU0QkVS 

---

target:0 (not a disaster)
Text
 Soul food sound so bomb right now ' 

---

target:0 (not a disaster)
Text
 Truth...
https://t.co/beJfTYgJIL
#News
#BBC
#CNN
#Islam
#Truth
#god
#ISIS
#terrorism
#Quran
#Lies http://t.co/jlCZiDZ7Vu 

---

target:0 (not a disaster)
Text
 Û¢Û¢If your lost &amp; alone or your sinking like a stone carry onå¡å¡ 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 A poignant reminder that in war there are many casualties. http://t.co/Mwmt3BdR5L       
 Vectorized version:
 [[   3 9708 3479   16    4  122   74   22  123  518    1    0    0    0
     0]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 70 years after #ABomb destroyd #HiroshimaÛÓ#BBC looks at wht #survived http://t.co/dLgNUuuUYn #CNV Watch Peace Vigils: http://t.co/jvkYzNDtja         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02037184,  0.04538629,  0.0159722 , ...,  0.00055263,
         -0.02498749,  0.03162665],
        [-0.04021464,  0.00536598,  0.0440514 , ..., -0.0410228 ,
          0.03617121, -0.04810555],
        [-0.04186716, -0.01770912, -0.00510329, ...,  0.01500878,
          0.02440924,  0.0236146 ],
        ...,
        [ 0.00813786,  0.01814431,  0.01408208, ..., -0.00132617,
          0.01982066,  0.00471684],
        [-0.01914491, -0.02445505, -0.03310698, ...,  0.02238691,
          0.02601549,  0.0111653 ],
        [-0.02548305,  0.00589212, -0.00112344, ...,  0.03657481,
          0.04350108,  0.04065558]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('70 years after #ABomb destroyd #Hiroshima\x89ÛÓ#BBC looks at wht #survived http://t.co/dLgNUuuUYn #CNV Watch Peace Vigils: http://t.co/jvkYzNDtja',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02037184,  0.04538629,  0.0159722 , -0.04636393, -0.00997959,
         0.01894924, -0.02516092,  0.0033538 , -0.03675685, -0.03423183,
        -0.04081406, -0.03534856, -0.04053333, -0.02417394, -0.00101168,
        -0.00052284,  0.04458532,  0.02349119, -0.02669247,  0.0043451 ,
        -0.03308593, -0.00978624, -0.02849152,  0.01492894,  0.03051731,
         0.0302048 ,  0.0454554 ,  0.01642543, -0.00204539,  0.0223627 ,
         0.0442646 ,  0.04251388, -0.0117023 , -0.03455322,  0.03404612,
         0.01682303, -0.03698646, -0.02402581,  0.01691074, -0.02567129,
         0.04475002, -0.0152479 , -0.04792403,  0.03221561, -0.02448152,
         0.04349909,  0.01738488,  0.01017585,  0.03849507,  0.00336747,
         0.0092749 , -0.04454483,  0.03421085,  0.03028179,  0.04434844

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220920-123725
Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.6157 - accuracy: 0.6875 - val_loss: 0.5385 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4421 - accuracy: 0.8187 - val_loss: 0.4668 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.3469 - accuracy: 0.8643 - val_loss: 0.4571 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 7s 30ms/step - loss: 0.2847 - accuracy: 0.8898 - val_loss: 0.4587 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.2378 - accuracy: 0.9132 - val_loss: 0.4773 - val_accuracy: 0.7861


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 7ms/step - loss: 0.4773 - accuracy: 0.7861


[0.47733697295188904, 0.7860892415046692]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.37965658],
       [0.7795496 ],
       [0.99758524],
       [0.10016447],
       [0.12104493],
       [0.9390328 ],
       [0.9220928 ],
       [0.9931001 ],
       [0.96162724],
       [0.23459053]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7860892388451444,
 'precision': 0.7911704681403405,
 'recall': 0.7860892388451444,
 'f1': 0.7830068921982195}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks

###Model 2 : LSTM

In [43]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [44]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [45]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220920-123801
Epoch 1/5
215/215 [==============================] - 26s 69ms/step - loss: 0.2204 - accuracy: 0.9222 - val_loss: 0.5843 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 9s 40ms/step - loss: 0.1568 - accuracy: 0.9435 - val_loss: 0.6780 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 7s 31ms/step - loss: 0.1276 - accuracy: 0.9526 - val_loss: 0.6934 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 7s 30ms/step - loss: 0.1063 - accuracy: 0.9585 - val_loss: 0.7543 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0840 - accuracy: 0.9683 - val_loss: 1.0140 - val_accuracy: 0.7743


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[3.2664239e-03],
        [4.6441853e-01],
        [9.9970627e-01],
        [1.7966658e-02],
        [3.8003922e-04],
        [9.9781442e-01],
        [9.2499089e-01],
        [9.9979627e-01],
        [9.9966550e-01],
        [7.1123511e-01]], dtype=float32))

In [47]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [48]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7742782152230971,
 'precision': 0.7757380419380466,
 'recall': 0.7742782152230971,
 'f1': 0.7723566516531356}

In [49]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation= 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name='model_3_GRU')

In [51]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [52]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data =(val_sentences, val_labels),
                        callbacks= [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220920-123934
Epoch 1/5
215/215 [==============================] - 10s 33ms/step - loss: 0.1551 - accuracy: 0.9413 - val_loss: 0.6943 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0863 - accuracy: 0.9679 - val_loss: 0.8577 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 7s 33ms/step - loss: 0.0711 - accuracy: 0.9717 - val_loss: 0.9815 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0565 - accuracy: 0.9755 - val_loss: 1.1613 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0545 - accuracy: 0.9755 - val_loss: 1.1806 - val_accuracy: 0.7690


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[8.5806549e-03],
       [8.1987119e-01],
       [9.9995887e-01],
       [2.6238608e-01],
       [1.9590860e-05],
       [9.9982476e-01],
       [9.2815018e-01],
       [9.9997616e-01],
       [9.9996269e-01],
       [5.7815564e-01]], dtype=float32)

In [54]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_3_results = calculate_result(y_true= val_labels,
                                   y_pred = model_3_preds)
model_3_results

{'accuracy': 0.7690288713910761,
 'precision': 0.7701356893081864,
 'recall': 0.7690288713910761,
 'f1': 0.7671875324347506}

### Model 4: Bidirectional RNN

In [56]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')

In [57]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [58]:
model_4.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [59]:
history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data = (val_sentences,val_labels),
                        callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                 experiment_name= 'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220920-124019
Epoch 1/5
215/215 [==============================] - 14s 44ms/step - loss: 0.1063 - accuracy: 0.9677 - val_loss: 0.9137 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0519 - accuracy: 0.9762 - val_loss: 1.3551 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 10s 46ms/step - loss: 0.0463 - accuracy: 0.9775 - val_loss: 1.1795 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0395 - accuracy: 0.9816 - val_loss: 1.6002 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 9s 40ms/step - loss: 0.0421 - accuracy: 0.9796 - val_loss: 1.3836 - val_accuracy: 0.7651


In [60]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.5047491e-03],
       [7.9321945e-01],
       [9.9996835e-01],
       [2.5936592e-01],
       [1.4876988e-05],
       [9.9977213e-01],
       [6.0493541e-01],
       [9.9998868e-01],
       [9.9996704e-01],
       [9.9928439e-01]], dtype=float32)

In [61]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
model_4_results = calculate_result(y_true= val_labels,
                                   y_pred = model_4_preds)
model_4_results

{'accuracy': 0.7650918635170604,
 'precision': 0.7658125387191,
 'recall': 0.7650918635170604,
 'f1': 0.7634031052086162}

### Model 5:conv1D

In [63]:
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(['this is a test sentence']))
conv_1d = layers.Conv1D(filters = 32,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

In [64]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters= 64, kernel_size=5,strides=1,activation= 'relu',padding= 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation= 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name= 'model_5_Conv1D')

model_5.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [65]:
history_5 = model_5.fit(train_sentences,
                        train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                 'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220920-124111
Epoch 1/5
215/215 [==============================] - 6s 22ms/step - loss: 0.1161 - accuracy: 0.9613 - val_loss: 0.9711 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0731 - accuracy: 0.9729 - val_loss: 1.0905 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0603 - accuracy: 0.9775 - val_loss: 1.2089 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0550 - accuracy: 0.9772 - val_loss: 1.2128 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0511 - accuracy: 0.9788 - val_loss: 1.2465 - val_accuracy: 0.7572


In [66]:
model_5_pred_probs=model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[9.4974637e-02],
       [4.9141094e-01],
       [9.9993312e-01],
       [6.9340676e-02],
       [6.1158183e-08],
       [9.9829513e-01],
       [9.7292519e-01],
       [9.9997282e-01],
       [9.9999845e-01],
       [8.4309244e-01]], dtype=float32)

In [67]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
model_5_results= calculate_result(y_true= val_labels,
                                  y_pred = model_5_preds)
model_5_results

{'accuracy': 0.7572178477690289,
 'precision': 0.758370695800474,
 'recall': 0.7572178477690289,
 'f1': 0.7550840315229969}

## Model 6: Tensorflow Hub pretrained sentence encoder

In [69]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "when you call the universal sentence encoder on sentence ,it turns it  into numbers "])
print(embed_sample[0][:50])

tf.Tensor(
[-0.00767769  0.03333809  0.0320662  -0.02125602  0.04300849  0.08399193
  0.02452653  0.05380183 -0.02588227 -0.00499588  0.01928363 -0.00056099
  0.01276441  0.09088546  0.0562213  -0.04887728  0.05191502 -0.04136144
 -0.01161554 -0.05452844 -0.01740017  0.0045459   0.00692502 -0.01381139
 -0.00961125 -0.03933032  0.0197613  -0.01115342 -0.03671153  0.01013187
 -0.04107149  0.03479797 -0.03814119 -0.00185953  0.01663749 -0.08907217
  0.00592126  0.04859087 -0.03741663 -0.08817419  0.00334023  0.00904682
 -0.01701778  0.06431309 -0.10647684 -0.0381299  -0.02107327 -0.02888192
 -0.03679271  0.01302323], shape=(50,), dtype=float32)


In [70]:
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape=[],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [71]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation= 'relu'),
    layers.Dense(1, activation = 'sigmoid',name= 'output_layer')
],name = 'model_6_USE')

model_6.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [72]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [73]:
history_6 = model_6.fit(train_sentences,
                        train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                                'tf_hub_sentence_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20220920-124159
Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.5021 - accuracy: 0.7902 - val_loss: 0.4434 - val_accuracy: 0.8071
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4140 - accuracy: 0.8130 - val_loss: 0.4363 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4007 - accuracy: 0.8218 - val_loss: 0.4327 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3940 - accuracy: 0.8246 - val_loss: 0.4308 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3878 - accuracy: 0.8275 - val_loss: 0.4265 - val_accuracy: 0.8110


In [74]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.19902733],
       [0.81139815],
       [0.98938394],
       [0.19102761],
       [0.7390057 ],
       [0.71856004],
       [0.98503464],
       [0.9782518 ],
       [0.94376975],
       [0.09415752]], dtype=float32)

In [75]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [76]:
model_6_results = calculate_result(y_true= val_labels,
                                y_pred = model_6_preds)
model_6_results

{'accuracy': 0.8110236220472441,
 'precision': 0.8122955784373107,
 'recall': 0.8110236220472441,
 'f1': 0.8098067287043664}

In [77]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7: TF hub pretrained USE but with 10 % of the data

In [78]:
# train_10_percent = train_df_shuffled[['text','target']].sample(frac=0.1, random_state = 42)
# train_sentences_10_percent = train_10_percent['text'].to_list()
# train_labels_10_percent = train_10_percent['target'].to_list()

# len(train_sentences_10_percent), len(train_labels_10_percent)

In [79]:
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [84]:
pd.Series(train_labels_10_percent).value_counts()

0    406
1    279
dtype: int64

In [85]:
# Let's build a model the same as model_6
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid", name="output_layer")                               
], name="model_7_USE")

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [86]:
history_7 = model_7.fit(train_sentences_10_percent,
                        train_labels_10_percent,
                        epochs = 5, 
                        validation_data = (val_sentences , val_labels),
                        callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                 'tf_hub_sentence_encoder_10_percent_correct_split')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20220920-124420
Epoch 1/5
22/22 [==============================] - 4s 76ms/step - loss: 0.6637 - accuracy: 0.6847 - val_loss: 0.6442 - val_accuracy: 0.6916
Epoch 2/5
22/22 [==============================] - 1s 38ms/step - loss: 0.5913 - accuracy: 0.7971 - val_loss: 0.5835 - val_accuracy: 0.7690
Epoch 3/5
22/22 [==============================] - 1s 50ms/step - loss: 0.5156 - accuracy: 0.8204 - val_loss: 0.5322 - val_accuracy: 0.7717
Epoch 4/5
22/22 [==============================] - 1s 30ms/step - loss: 0.4561 - accuracy: 0.8350 - val_loss: 0.5000 - val_accuracy: 0.7782
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.4174 - accuracy: 0.8394 - val_loss: 0.4877 - val_accuracy: 0.7835


In [87]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.20406476],
       [0.58549905],
       [0.90789723],
       [0.40280357],
       [0.59031916],
       [0.6773936 ],
       [0.8732029 ],
       [0.82237375],
       [0.8702433 ],
       [0.15888566]], dtype=float32)

In [88]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [89]:
model_7_results = calculate_result(y_true = val_labels,
                                    y_pred = model_7_preds)
model_7_results

{'accuracy': 0.7834645669291339,
 'precision': 0.7839822035668361,
 'recall': 0.7834645669291339,
 'f1': 0.782225309795662}

In [90]:
model_6_results

{'accuracy': 0.8110236220472441,
 'precision': 0.8122955784373107,
 'recall': 0.8110236220472441,
 'f1': 0.8098067287043664}